In [ ]:
%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)

TensorFlow 1.x selected.
1.15.2


In [ ]:
!pip3 install sentencepiece
!pip3 install tf-sentencepiece

     |████████████████████████████████| 1.1MB 9.0MB/s 
     |████████████████████████████████| 2.1MB 8.6MB/s 


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tf_sentencepiece
import pandas as pd
from google.colab import drive
from sys import path
import math

In [ ]:
root = '/content/drive/My Drive/nlp-lab'


#processed_data_location = root + '/data/processed-data/amazon-english'
processed_data_location = root + '/data/processed-data/amazon-german-dataset2'
#processed_data_location = root + '/data/processed-data/organic'

exported_file_name = 'processed_data_with_embeddings_XLING.json'

In [ ]:
drive.mount('/content/drive', force_remount=True)
path.append(root)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
data_df = pd.read_json(processed_data_location + '/processed_data_without_embeddings_no_outliers.json')

In [ ]:
sentences = data_df['sentence_text'].tolist() 

In [ ]:
# Set up graph.
g = tf.Graph()
with g.as_default():
  text_input = tf.placeholder(dtype=tf.string, shape=[None])
  xling_8_embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-xling-many/1")
  embedded_text = xling_8_embed(text_input)
  init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
# Initialize session.
session = tf.Session(graph=g)
session.run(init_op)

In [ ]:
# Compute embeddings.
chunk_size= 1000
sentences_encoded = np.empty([0,512]) 

for i in range(0, len(sentences), chunk_size):
    print('generating embeddings for chunk: ' + str(math.floor(i/chunk_size)) + "/" + str(math.floor(len(sentences)/chunk_size)))
    chunk = sentences[i:i+chunk_size]
    chunk_encoded= session.run(embedded_text, feed_dict={text_input: chunk})
    sentences_encoded = np.concatenate((sentences_encoded,chunk_encoded),axis=0)

generating encodings for chunk: 0/28
generating encodings for chunk: 1/28
generating encodings for chunk: 2/28
generating encodings for chunk: 3/28
generating encodings for chunk: 4/28
generating encodings for chunk: 5/28
generating encodings for chunk: 6/28
generating encodings for chunk: 7/28
generating encodings for chunk: 8/28
generating encodings for chunk: 9/28
generating encodings for chunk: 10/28
generating encodings for chunk: 11/28
generating encodings for chunk: 12/28
generating encodings for chunk: 13/28
generating encodings for chunk: 14/28
generating encodings for chunk: 15/28
generating encodings for chunk: 16/28
generating encodings for chunk: 17/28
generating encodings for chunk: 18/28
generating encodings for chunk: 19/28
generating encodings for chunk: 20/28
generating encodings for chunk: 21/28
generating encodings for chunk: 22/28
generating encodings for chunk: 23/28
generating encodings for chunk: 24/28
generating encodings for chunk: 25/28
generating encodings f

In [ ]:
data_with_embeddings = pd.concat([data_df.reset_index(drop=True),
                                  pd.DataFrame(sentences_encoded)], axis=1)

In [ ]:
data_with_embeddings.to_json(processed_data_location + '/' + exported_file_name)

/content/drive/My Drive/nlp-lab/data/processed-data/amazon-german-dataset2/processed_data_with_embeddings_XLING.json
